<a href="https://colab.research.google.com/github/mostafa-ja/Transformers/blob/main/Self_supervised_Federated_Learning_(SSL_FL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self-supervised Federated Learning (SSL-FL)

[link text](https://github.com/rui-yan/SSL-FL)

In [ ]:
!pip install timm=="0.3.2"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 KB 13.6 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/rui-yan/SSL-FL.git

Cloning into 'SSL-FL'...
remote: Enumerating objects: 2316, done.
remote: Counting objects: 100% (590/590), done.
remote: Compressing objects: 100% (311/311), done.
remote: Total 2316 (delta 345), reused 436 (delta 272), pack-reused 1726
Receiving objects: 100% (2316/2316), 15.70 MiB | 17.86 MiB/s, done.
Resolving deltas: 100% (1022/1022), done.


In [ ]:
!pip install gdown
import gdown
!gdown https://drive.google.com/u/0/uc?id=1bW--_qRZnWbkb0XXvGBCSferdqXZ6pe7&export=download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/u/0/uc?id=1bW--_qRZnWbkb0XXvGBCSferdqXZ6pe7
To: /content/Retina.zip
100% 4.50G/4.50G [01:10<00:00, 64.0MB/s]


In [ ]:
!gdown https://drive.google.com/u/0/uc?id=18cG2rrweNKc8LS5LBTcUAt9A4om3YWGz&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=18cG2rrweNKc8LS5LBTcUAt9A4om3YWGz
To: /content/retina_pretrain_mae_base_split1_checkpoint-1599.pth
100% 1.34G/1.34G [00:24<00:00, 55.0MB/s]


In [ ]:
!unzip "/content/Retina.zip" -d "/content/SSL-FL/data/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/SSL-FL/data/Retina/train/8732_right_test.npy  
  inflating: /content/SSL-FL/data/Retina/train/41353_right.npy  
  inflating: /content/SSL-FL/data/Retina/train/8909_right_test.npy  
  inflating: /content/SSL-FL/data/Retina/train/16148_right.npy  
  inflating: /content/SSL-FL/data/Retina/train/15511_right_test.npy  
  inflating: /content/SSL-FL/data/Retina/train/31753_right_test.npy  
  inflating: /content/SSL-FL/data/Retina/train/12868_right.npy  
  inflating: /content/SSL-FL/data/Retina/train/30622_right_test.npy  
  inflating: /content/SSL-FL/data/Retina/train/39518_right.npy  
  inflating: /content/SSL-FL/data/Retina/train/12890_right_test.npy  
  inflating: /content/SSL-FL/data/Retina/train/31594_right.npy  
  inflating: /content/SSL-FL/data/Retina/train/21520_right.npy  
  inflating: /content/SSL-FL/data/Retina/train/32392_right_test.npy  
  inflating: /content/SSL-FL/data/Retina/train/31149_right_test.npy  
 

In [ ]:
FED_MODEL='fed_mae'

%cd /content/SSL-FL/code/{FED_MODEL}/ 
!pwd

/content/SSL-FL/code/fed_mae
/content/SSL-FL/code/fed_mae


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0




```
/usr/local/lib/python3.9/dist-packages/timm/models/layers/helpers.py
/usr/local/lib/python3.9/dist-packages/timm/models/layers/helpers.py

import torch
TORCH_MAJOR = int(torch.__version__.split('.')[0])
TORCH_MINOR = int(torch.__version__.split('.')[1])

if TORCH_MAJOR == 1 and TORCH_MINOR < 8:
    from torch._six import container_abcs
else:
    import collections.abc as container_abcs



# /content/SSL-FL/code/util/misc.py
from torch._six import inf >> from torch._six import inf
```



In [19]:
!pip install einops
from einops import rearrange

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
#!/bin/bash
# This script run Fed-MAE pre-training on Retina Split-1 dataset, then finetune
# Please modify the path DATA_PATH, OUTPUT_PATH

FED_MODEL='fed_mae'

%cd /content/SSL-FL/code/{FED_MODEL}/

DATASET='Retina' # dataset name
DATA_PATH="/content/SSL-FL/data/{DATASET}/" # change DATA_PATH to the path where the data were stored

SPLIT_TYPE='split_1' # chosen from {'split_1', 'split_2', 'split_3'}
N_CLASSES=2 # the number of classes in the dataset
N_CLIENTS=5 # number of clients in the federated setting
MASK_RATIO=0.6 # masking ratio for Fed-BEiT pre-training
N_GPUS=1 # the number of GPUs used for model training

# ------------------ finetune ----------------- #
# CKPT_PATH="${OUTPUT_PATH}/checkpoint-1599.pth"
CKPT_PATH="/content/retina_pretrain_mae_base_split1_checkpoint-1599.pth"
FT_EPOCHS=100
FT_LR='3e-3'
FT_BATCH_SIZE=64
OUTPUT_PATH_FT="{OUTPUT_PATH}/finetune_{DATASET}_epoch{FT_EPOCHS}_{SPLIT_TYPE}_lr{FT_LR}_bs{FT_BATCH_SIZE}_dis{N_GPUS}"

# change the CUDA devices available for model training 
!CUDA_VISIBLE_DEVICES=0 OMP_NUM_THREADS=1 python -m torch.distributed.launch --nproc_per_node={N_GPUS} run_class_finetune_FedAvg.py \
     --data_path {DATA_PATH} \
     --data_set {DATASET} \
     --finetune {CKPT_PATH} \
     --nb_classes {N_CLASSES} \
     --output_dir {OUTPUT_PATH_FT} \
     --save_ckpt_freq 50 \
     --model vit_base_patch16 \
     --batch_size {FT_BATCH_SIZE} --split_type ${SPLIT_TYPE} \
     --blr ${FT_LR} --layer_decay 0.65 \
     --weight_decay 0.05 --drop_path 0.1 --reprob 0.25 --mixup 0.8 --cutmix 1.0 \
     --n_clients {N_CLIENTS} --E_epoch 1 --max_communication_rounds {FT_EPOCHS} --num_local_clients -1

/content/SSL-FL/code/fed_mae
/usr/local/lib/python3.9/dist-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
2023-04-05 18:51:52.189746: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 18:51:53.049101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/SSL-FL/code/f